In [ ]:
!pip install -U bitsandbytes sentencepiece protobuf

In [2]:
from huggingface_hub import login
from google.colab import userdata

token_value = userdata.get('hf_token')
login(token=token_value)

In [1]:
from sklearn.metrics.pairwise import cosine_distances
import numpy as np

def select_best_layer_cosine(activations: np.ndarray, labels: np.ndarray) -> tuple:
    """
    Визначає найкращий шар нейронної мережі на основі косинусної відстані між активаціями двох класів.

    Аналізує активації для кожного шару та обчислює середню косинусну відстань між активаціями
    нейтрального класу (labels=0) та спеціального класу (labels=1). Найкращим вважається шар,
    де спостерігається найбільше зростання цієї відстані порівняно з попереднім шаром.

    Параметри:
    ----------
    activations : np.ndarray
        3D-масив активацій нейронної мережі, форма (n_samples, n_layers, hidden_size).
        Містить активації для кожного зразка, шару та нейрона прихованого шару.
    labels : np.ndarray
        1D-масив міток класів (0 - нейтральний клас, 1 - спеціальний клас).

    Повертає:
    -------
    tuple
        best_layer : int
            Номер шару з найбільшим приростом косинусної відстані.
        layer_distances : dict
            Словник із середніми косинусними відстанями для кожного шару {layer: mean_distance}.
        layer_deltas : dict
            Словник із різницями відстаней між поточним та попереднім шаром {layer: delta}.
    """
    n_samples, n_layers, hidden_size = activations.shape
    layer_distances = {}
    layer_deltas = {0: 0}
    
    neutral_acts = activations[labels == 0]
    traited_acts = activations[labels == 1]

    assert len(neutral_acts) == len(traited_acts), "Кількість зразків у класах не співпадає"

    # Обчислення середньої косинусної відстані для кожного шару
    for layer in range(n_layers):
        neutral_layer = neutral_acts[:, layer, :]
        traited_layer = traited_acts[:, layer, :]
        
        distances = cosine_distances(neutral_layer, traited_layer)
        mean_distance = np.mean(np.diag(distances))
        layer_distances[layer] = mean_distance

    # Обчислення дельт (різниць між шарами)
    for layer in range(1, n_layers):
        delta = layer_distances[layer] - layer_distances[layer-1]
        layer_deltas[layer] = delta

    best_layer = max(layer_deltas, key=layer_deltas.get)
    return best_layer, layer_distances, layer_deltas

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np

def select_best_layer_logreg(activations: np.ndarray, labels: np.ndarray) -> tuple:
    """
    Визначає найкращий шар нейронної мережі на основі AUC-ROC логістичної регресії.

    Для кожного шару активацій навчає модель логістичної регресії та обчислює AUC-ROC,
    щоб оцінити, наскільки добре активації шару лінійно розділяють два класи. 
    Найкращим вважається шар із найвищим значенням AUC-ROC.

    Параметри:
    ----------
    activations : np.ndarray
        3D-масив активацій нейронної мережі, форма (n_samples, n_layers, hidden_size).
        Містить активації для кожного зразка, шару та нейрона прихованого шару.
    labels : np.ndarray
        1D-масив міток класів (0 - перший клас, 1 - другий клас).

    Повертає:
    -------
    tuple
        best_layer : int
            Номер шару з найвищим значенням AUC-ROC.
        layer_aucs : dict
            Словник із значеннями AUC-ROC для кожного шару {layer: auc_score}.
    """
    n_samples, n_layers, hidden_size = activations.shape
    layer_aucs = {}

    for layer in range(n_layers):
        X_layer = activations[:, layer, :]
        clf = LogisticRegression(max_iter=1000)
        clf.fit(X_layer, labels)

        probs = clf.predict_proba(X_layer)[:, 1]
        pred = clf.predict(X_layer)
        auc = roc_auc_score(labels, probs)
        layer_aucs[layer] = auc

    best_layer = max(layer_aucs, key=layer_aucs.get)
    return best_layer, layer_aucs

In [8]:
import json
from tqdm import tqdm
from typing import Tuple
import numpy as np
import torch


def collect_activations(
    model,
    tokenizer,
    dataset_path: str,
    model_type: str = "llama",  # "llama", "mistral", "flan_t5"
    max_samples: int = None,
    batch_size: int = 8
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Збирає активації з прихованих шарів моделі (включаючи енкодер і декодер для T5) батчами.

    Параметри:
    ----------
    model : PreTrainedModel
        Модель трансформера для генерації та отримання активацій.
    tokenizer : PreTrainedTokenizer
        Токенізатор для обробки вхідних текстів.
    dataset_path : str
        Шлях до JSON-файлу з датасетом.
    model_type : str
        Тип моделі: "llama", "mistral" — автогресивні; "flan_t5" — енкодер-декодер.
    max_samples : int, optional
        Максимальна кількість зразків для обробки.
    batch_size : int, optional
        Розмір батчу.

    Повертає:
    --------
    Tuple[np.ndarray, np.ndarray]
        all_activations : np.ndarray
            3D-масив активацій форми (n_samples, n_layers, hidden_size).
        all_labels : np.ndarray
            1D-масив міток (0 - нейтральний, 1 - спеціальний).
    """
    with open(dataset_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    if max_samples is not None:
        data = data[:max_samples]

    prompts = []
    labels = []

    for sample in data:
        prompt_pair = sample["prompts"][model_type]
        prompts.extend([prompt_pair["neutral"], prompt_pair["traited"]])
        labels.extend([0, 1])

    all_activations = []
    all_labels = []

    for i in tqdm(range(0, len(prompts), batch_size), desc="Обробка батчів"):
        batch_prompts = prompts[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]

        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        with torch.no_grad():
            if model_type == "flan_t5":
                decoder_input_ids = model.prepare_decoder_input_ids_from_labels(inputs["input_ids"])
                outputs = model(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    decoder_input_ids=decoder_input_ids,
                    output_hidden_states=True,
                    return_dict=True
                )

                # Енкодер
                encoder_hidden = torch.stack(outputs.encoder_hidden_states[1:])  # (n_layers, batch, seq_len, hidden)
                enc_mask = inputs["attention_mask"].unsqueeze(0).unsqueeze(-1)  # (1, batch, seq_len, 1)
                enc_sum = (encoder_hidden * enc_mask).sum(dim=2)
                enc_div = enc_mask.sum(dim=2)
                enc_activations = (enc_sum / enc_div)  # (n_layers, batch, hidden)

                # Декодер
                decoder_hidden = torch.stack(outputs.decoder_hidden_states[1:])  # (n_layers, batch, seq_len, hidden)
                dec_mask = (decoder_input_ids != tokenizer.pad_token_id).unsqueeze(0).unsqueeze(-1)
                dec_sum = (decoder_hidden * dec_mask).sum(dim=2)
                dec_div = dec_mask.sum(dim=2)
                dec_activations = (dec_sum / dec_div)  # (n_layers, batch, hidden)

                # Об'єднання енкодера та декодера
                combined = torch.cat([enc_activations, dec_activations], dim=0)  # (2 * n_layers, batch, hidden)
                layer_activations = combined.permute(1, 0, 2).cpu().numpy()  # (batch, 2 * n_layers, hidden)

            else:
                outputs = model(**inputs, output_hidden_states=True, return_dict=True)
                hidden_states = torch.stack(outputs.hidden_states[1:])  # (n_layers, batch, seq_len, hidden)
                mask = inputs["attention_mask"].unsqueeze(0).unsqueeze(-1)  # (1, batch, seq_len, 1)
                sum_states = (hidden_states * mask).sum(dim=2)
                sum_mask = mask.sum(dim=2)
                layer_activations = (sum_states / sum_mask).permute(1, 0, 2).cpu().numpy()  # (batch, n_layers, hidden)

        all_activations.append(layer_activations)
        all_labels.extend(batch_labels)

    all_activations = np.concatenate(all_activations, axis=0)
    all_labels = np.array(all_labels)

    return all_activations, all_labels

In [4]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig
)
import torch
from typing import Optional

def load_model(model_name: str, bnb_config: Optional[BitsAndBytesConfig] = None):
    """
    Завантажує модель (LLaMA, Mistral або Flan-T5) з Hugging Face Hub з підтримкою 4/8-бітної квантізації.

    Параметри:
        model_name (str): Назва моделі на Hugging Face Hub.
        bnb_config (BitsAndBytesConfig, optional): Об'єкт конфігурації квантізації BitsAndBytes. 
                                                   Якщо None — модель буде завантажена без квантізації або у float16.

    Повертає:
        tuple: (model, tokenizer) — завантажена модель та відповідний токенізатор.
    """
    torch.cuda.empty_cache()

    is_seq2seq = "flan-t5" in model_name.lower() or "t5" in model_name.lower()

    if bnb_config is not None:
        model_class = AutoModelForSeq2SeqLM if is_seq2seq else AutoModelForCausalLM
        model = model_class.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            output_hidden_states=True,
            return_dict_in_generate=True
        )
    else:
        model_class = AutoModelForSeq2SeqLM if is_seq2seq else AutoModelForCausalLM
        model = model_class.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16,
            output_hidden_states=True,
            return_dict_in_generate=True
        )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token
    tokenizer.padding_side = "left"

    model.eval()
    return model, tokenizer

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

In [6]:
llama_model, llama_tokenizer = load_model("meta-llama/Llama-2-7b-hf", bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
generation_args = {
    "max_new_tokens": 256,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.9
}

all_activations, all_labels = collect_activations(llama_model, llama_tokenizer, "trait_combined_dataset.json", "llama", batch_size=8)

Обробка батчів: 100%|██████████| 2275/2275 [16:48<00:00,  2.26it/s] 


In [8]:
all_activations.shape, all_labels.shape

((18200, 32, 4096), (18200,))

In [9]:
best_layer, layer_distances, layers_deltas = select_best_layer_cosine(all_activations, all_labels)

print("Layers scores:")
for layer, distance in layer_distances.items():
    print(f"Layer {layer}: score = {distance:.4f}, delta = {layers_deltas[layer]:.4f}")

print(f"\nBest layer: {best_layer} with score = {layer_distances[best_layer]:.4f} delta = {layers_deltas[best_layer]:.4f}")

Layers scores:
Layer 0: score = 0.0142, delta = 0.0000
Layer 1: score = 0.0001, delta = -0.0141
Layer 2: score = 0.0002, delta = 0.0001
Layer 3: score = 0.0004, delta = 0.0002
Layer 4: score = 0.0006, delta = 0.0002
Layer 5: score = 0.0008, delta = 0.0003
Layer 6: score = 0.0012, delta = 0.0004
Layer 7: score = 0.0017, delta = 0.0005
Layer 8: score = 0.0022, delta = 0.0005
Layer 9: score = 0.0028, delta = 0.0005
Layer 10: score = 0.0035, delta = 0.0008
Layer 11: score = 0.0043, delta = 0.0008
Layer 12: score = 0.0053, delta = 0.0010
Layer 13: score = 0.0069, delta = 0.0017
Layer 14: score = 0.0081, delta = 0.0011
Layer 15: score = 0.0107, delta = 0.0027
Layer 16: score = 0.0163, delta = 0.0056
Layer 17: score = 0.0197, delta = 0.0034
Layer 18: score = 0.0259, delta = 0.0061
Layer 19: score = 0.0323, delta = 0.0064
Layer 20: score = 0.0425, delta = 0.0102
Layer 21: score = 0.0492, delta = 0.0067
Layer 22: score = 0.0598, delta = 0.0107
Layer 23: score = 0.0632, delta = 0.0033
Layer 24: 

In [10]:
best_layer, layer_aucs = select_best_layer_logreg(all_activations, all_labels)

print("Layers scores:")
for layer, score in layer_aucs.items():
    print(f"Layer {layer}: score = {score:.4f}")

print(f"\nBest layer: {best_layer} with score = {layer_aucs[best_layer]:.4f}")


Layers scores:
Layer 0: score = 0.9998
Layer 1: score = 1.0000
Layer 2: score = 1.0000
Layer 3: score = 1.0000
Layer 4: score = 1.0000
Layer 5: score = 1.0000
Layer 6: score = 1.0000
Layer 7: score = 1.0000
Layer 8: score = 1.0000
Layer 9: score = 1.0000
Layer 10: score = 1.0000
Layer 11: score = 1.0000
Layer 12: score = 1.0000
Layer 13: score = 1.0000
Layer 14: score = 1.0000
Layer 15: score = 1.0000
Layer 16: score = 1.0000
Layer 17: score = 1.0000
Layer 18: score = 1.0000
Layer 19: score = 1.0000
Layer 20: score = 1.0000
Layer 21: score = 1.0000
Layer 22: score = 1.0000
Layer 23: score = 1.0000
Layer 24: score = 1.0000
Layer 25: score = 1.0000
Layer 26: score = 1.0000
Layer 27: score = 1.0000
Layer 28: score = 1.0000
Layer 29: score = 1.0000
Layer 30: score = 1.0000
Layer 31: score = 1.0000

Best layer: 3 with score = 1.0000


In [6]:
mistral_model, mistral_tokenizer = load_model("mistralai/Mistral-7B-v0.3", bnb_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
generation_args = {
    "max_new_tokens": 256,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.9
}

all_activations, all_labels = collect_activations(mistral_model, mistral_tokenizer, "trait_combined_dataset.json", "mistral")

Обробка батчів: 100%|██████████| 2275/2275 [15:48<00:00,  2.40it/s]


In [8]:
all_activations.shape, all_labels.shape

((18200, 32, 4096), (18200,))

In [9]:
best_layer, layer_distances, layers_deltas = select_best_layer_cosine(all_activations, all_labels)

print("Layers scores:")
for layer, distance in layer_distances.items():
    print(f"Layer {layer}: score = {distance:.4f}, delta = {layers_deltas[layer]:.4f}")

print(f"\nBest layer: {best_layer} with score = {layer_distances[best_layer]:.4f} delta = {layers_deltas[best_layer]:.4f}")

Layers scores:
Layer 0: score = 0.1329, delta = 0.0000
Layer 1: score = 0.0200, delta = -0.1129
Layer 2: score = 0.0199, delta = -0.0000
Layer 3: score = 0.0200, delta = 0.0001
Layer 4: score = 0.0202, delta = 0.0002
Layer 5: score = 0.0209, delta = 0.0007
Layer 6: score = 0.0215, delta = 0.0006
Layer 7: score = 0.0226, delta = 0.0011
Layer 8: score = 0.0230, delta = 0.0004
Layer 9: score = 0.0245, delta = 0.0015
Layer 10: score = 0.0250, delta = 0.0005
Layer 11: score = 0.0260, delta = 0.0010
Layer 12: score = 0.0278, delta = 0.0018
Layer 13: score = 0.0279, delta = 0.0002
Layer 14: score = 0.0294, delta = 0.0014
Layer 15: score = 0.0333, delta = 0.0039
Layer 16: score = 0.0389, delta = 0.0056
Layer 17: score = 0.0408, delta = 0.0019
Layer 18: score = 0.0502, delta = 0.0094
Layer 19: score = 0.0589, delta = 0.0087
Layer 20: score = 0.0675, delta = 0.0086
Layer 21: score = 0.0715, delta = 0.0040
Layer 22: score = 0.0733, delta = 0.0018
Layer 23: score = 0.0754, delta = 0.0021
Layer 24:

In [10]:
best_layer, layer_aucs = select_best_layer_logreg(all_activations, all_labels)

print("Layers scores:")
for layer, score in layer_aucs.items():
    print(f"Layer {layer}: score = {score:.4f}")

print(f"\nBest layer: {best_layer} with score = {layer_aucs[best_layer]:.4f}")

Layers scores:
Layer 0: score = 0.9860
Layer 1: score = 0.9954
Layer 2: score = 0.9991
Layer 3: score = 1.0000
Layer 4: score = 1.0000
Layer 5: score = 1.0000
Layer 6: score = 1.0000
Layer 7: score = 1.0000
Layer 8: score = 1.0000
Layer 9: score = 1.0000
Layer 10: score = 1.0000
Layer 11: score = 1.0000
Layer 12: score = 1.0000
Layer 13: score = 1.0000
Layer 14: score = 1.0000
Layer 15: score = 1.0000
Layer 16: score = 1.0000
Layer 17: score = 1.0000
Layer 18: score = 1.0000
Layer 19: score = 1.0000
Layer 20: score = 1.0000
Layer 21: score = 1.0000
Layer 22: score = 1.0000
Layer 23: score = 1.0000
Layer 24: score = 1.0000
Layer 25: score = 1.0000
Layer 26: score = 1.0000
Layer 27: score = 1.0000
Layer 28: score = 1.0000
Layer 29: score = 1.0000
Layer 30: score = 1.0000
Layer 31: score = 1.0000

Best layer: 8 with score = 1.0000


In [6]:
t5_model, t5_tokenizer = load_model("google/flan-t5-xl", bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [9]:
generation_args = {
    "max_new_tokens": 512,
    "do_sample": True,
    "temperature": 1.1,
    "top_p": 0.99,
    "top_k": 50
}

all_activations, all_labels = collect_activations(t5_model, t5_tokenizer, "trait_combined_dataset.json", "flan_t5")

Обробка батчів: 100%|██████████| 2275/2275 [08:23<00:00,  4.52it/s]


In [10]:
all_activations.shape, all_labels.shape

((18200, 48, 2048), (18200,))

In [11]:
best_layer, layer_distances, layers_deltas = select_best_layer_cosine(all_activations, all_labels)

print("Layers scores:")
for layer, distance in layer_distances.items():
    print(f"Layer {layer}: score = {distance:.4f}, delta = {layers_deltas[layer]:.4f}")

print(f"\nBest layer: {best_layer} with score = {layer_distances[best_layer]:.4f} delta = {layers_deltas[best_layer]:.4f}")

Layers scores:
Layer 0: score = 0.0390, delta = 0.0000
Layer 1: score = 0.0009, delta = -0.0381
Layer 2: score = 0.0011, delta = 0.0002
Layer 3: score = 0.0013, delta = 0.0002
Layer 4: score = 0.0015, delta = 0.0001
Layer 5: score = 0.0002, delta = -0.0013
Layer 6: score = 0.0002, delta = 0.0000
Layer 7: score = 0.0002, delta = 0.0000
Layer 8: score = 0.0002, delta = 0.0000
Layer 9: score = 0.0003, delta = 0.0000
Layer 10: score = 0.0004, delta = 0.0001
Layer 11: score = 0.0004, delta = 0.0001
Layer 12: score = 0.0005, delta = 0.0001
Layer 13: score = 0.0006, delta = 0.0001
Layer 14: score = 0.0008, delta = 0.0002
Layer 15: score = 0.0009, delta = 0.0001
Layer 16: score = 0.0013, delta = 0.0003
Layer 17: score = 0.0016, delta = 0.0003
Layer 18: score = 0.0019, delta = 0.0003
Layer 19: score = 0.0023, delta = 0.0004
Layer 20: score = 0.0027, delta = 0.0004
Layer 21: score = 0.0029, delta = 0.0003
Layer 22: score = 0.0031, delta = 0.0002
Layer 23: score = 0.2540, delta = 0.2509
Layer 24:

23 - останнiй шар енкодера

In [12]:
best_layer, layer_aucs = select_best_layer_logreg(all_activations, all_labels)

print("Layers scores:")
for layer, score in layer_aucs.items():
    print(f"Layer {layer}: score = {score:.4f}")

print(f"\nBest layer: {best_layer} with score = {layer_aucs[best_layer]:.4f}")

Layers scores:
Layer 0: score = 1.0000
Layer 1: score = 1.0000
Layer 2: score = 1.0000
Layer 3: score = 1.0000
Layer 4: score = 1.0000
Layer 5: score = 1.0000
Layer 6: score = 1.0000
Layer 7: score = 1.0000
Layer 8: score = 1.0000
Layer 9: score = 1.0000
Layer 10: score = 1.0000
Layer 11: score = 1.0000
Layer 12: score = 1.0000
Layer 13: score = 1.0000
Layer 14: score = 1.0000
Layer 15: score = 1.0000
Layer 16: score = 1.0000
Layer 17: score = 1.0000
Layer 18: score = 1.0000
Layer 19: score = 1.0000
Layer 20: score = 1.0000
Layer 21: score = 1.0000
Layer 22: score = 1.0000
Layer 23: score = 1.0000
Layer 24: score = 1.0000
Layer 25: score = 1.0000
Layer 26: score = 1.0000
Layer 27: score = 1.0000
Layer 28: score = 1.0000
Layer 29: score = 1.0000
Layer 30: score = 1.0000
Layer 31: score = 1.0000
Layer 32: score = 1.0000
Layer 33: score = 1.0000
Layer 34: score = 1.0000
Layer 35: score = 1.0000
Layer 36: score = 1.0000
Layer 37: score = 1.0000
Layer 38: score = 1.0000
Layer 39: score = 1.